# Collecting Tweets using Tweepy

In [1]:
# Load packages
import tweepy as tw
import pandas as pd
import numpy as np
from preprocessor import api as p #to clean tweets

In [2]:
# Credentials
access_token = "1119973522435035136-6FjyeGe4XlblWuaNNTIzwhjoNBxsrA"
access_token_secret = "NNYIIsZtlZc6YJpgbEbPWl9ecPfotympspjIIjrXVjWEp"
api_key = "A3koB4EuhkEfST1fcYizS2CHY"
api_key_secret = "WHo7GMPKTCJWNDmgaMBfsfZNFuLFR4Kp16aBwQJS7tKaSsnBHQ"

In [3]:
# Authentication app
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [18]:
# Find by phrase or keywords within a specified time
search_words = '"utah lake" -filter:links -filter:retweets' #filter out retweets and tweets with links
date_since = '2020-11-21'
date_until = '2020-11-28'

# Create a cursor object that iterates over the requested information
tweets = tw.Cursor(api.search, q=search_words, lang="en",
                  since=date_since, until=date_until, 
                  tweet_mode='extended').items()

#when we have premium subscription we will use this
#API.search_full_archive(environment_name, query[, tag][, fromDate][, toDate][, maxResults][, next])

# Parse the cursor object into rows of lists
rows = [[tweet.user.location, tweet.full_text, tweet.created_at] for tweet in tweets]

In [19]:
# Create dataframe from the rows of lists
df_tweet = pd.DataFrame(data=rows, columns=['location', 'tweet', 'time'])

In [20]:
# Add ID column (because we removed user information)
unique_id = np.arange(len(df_tweet))
df_tweet['id'] = unique_id + 1
df_tweet = df_tweet[['id', 'location', 'tweet', 'time']]

In [21]:
df_tweet.head()

,id,location,tweet,time
0,1,UK,November 28th\n\nNational Not ‘so long Sucker’...,2020-11-27 22:13:01
1,2,,"@jhallito @amccullough105 Honestly, having Veg...",2020-11-25 17:44:10
2,3,,@nameschip Is going to Utah lake at sunset in ...,2020-11-25 04:31:52
3,4,The Beehive State,@avery_abe This take is worse than going to Ut...,2020-11-25 04:25:09
4,5,UTAH,@wyattjarvis08 Dang! Heard about that. Lost ha...,2020-11-24 22:31:31


In [22]:
df_tweet.tail()

,id,location,tweet,time
5,6,"Washington, DC",@nilocobau @A320Lga Given the geography of the...,2020-11-24 13:47:00
6,7,TROLLEY SQUARE DUMPSTER,@torbenbernhard @GovHerbert needs to go jump i...,2020-11-23 23:27:53
7,8,In poasi’s bed,"@WGaisoa Mannn we live next to the Utah lake, ...",2020-11-23 03:32:00
8,9,California. Bitch.,@fuckoff1991ae @MidgetTamer I watched insidiou...,2020-11-21 22:38:20
9,10,,@SBoojum @Bossynotbossy I went to some really ...,2020-11-21 05:03:42


In [9]:
type(tweets)

tweepy.cursor.ItemIterator

# Cleaning tweets in the dataframe 

In [10]:
# Clean each tweet in our dataframe using the preprocessor module
tweet_clean = []
for tweet in df_tweet['tweet']:
    cleaned = p.clean(tweet)
    tweet_clean.append(cleaned)

In [11]:
# Store as a new dataframe
df_tweet_clean = df_tweet.copy()
df_tweet_clean['tweet'] = tweet_clean

In [12]:
df_tweet_clean.head()

,id,location,tweet,time
0,1,,Damn I was really disrespecting Utah once you ...,2020-11-25 23:37:16
1,2,,"'35.09""N '58.26""W based on the October data, i...",2020-11-25 23:04:02
2,3,"Salt Lake City, UT",Utah County from what I have had is kind of th...,2020-11-25 22:41:20
3,4,"Salt Lake City, UT",Called earlier today: I just wanted to call to...,2020-11-25 22:30:53
4,5,,I can see the dashboard now in an investor pre...,2020-11-25 22:28:59


# Save the dataframe as a CSV

In [24]:
# Checking current working directory
import os
os.getcwd()

'C:\\Users\\Peaceful Pierre\\Documents\\Academics\\Masters Project\\code'

In [26]:
# Store as CSV to a relative path
df_tweet_clean.to_csv("scraped_tweets_test.csv")